In [279]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [280]:
## get tweets from our repo
data = pd.read_csv(r"../inputdata/cnnhealth.txt", sep="|", header=None, on_bad_lines='skip')

In [281]:
print("Data in its unedited state: ")
print(data)

Data in its unedited state: 
                       0                               1  \
0     576880531301801984  Sat Mar 14 23:00:11 +0000 2015   
1     576820122666471424  Sat Mar 14 19:00:08 +0000 2015   
2     576744652717461504  Sat Mar 14 14:00:15 +0000 2015   
3     576736754436304896  Sat Mar 14 13:28:52 +0000 2015   
4     576736614766010368  Sat Mar 14 13:28:18 +0000 2015   
...                  ...                             ...   
4040  239699936671854593  Sun Aug 26 12:24:52 +0000 2012   
4041  239504620710420480  Sat Aug 25 23:28:46 +0000 2012   
4042  239410205757145088  Sat Aug 25 17:13:35 +0000 2012   
4043  239386320416428032  Sat Aug 25 15:38:41 +0000 2012   
4044  239366825018806272  Sat Aug 25 14:21:12 +0000 2012   

                                                      2  
0     An abundance of online info can turn us into e...  
1     A plant-based diet that incorporates fish may ...  
2     It doesn't take much to damage your hearing at...  
3     RT @CNN: For

In [282]:
## Removing urls from our tweets. One potential way to remove non-meaningful information such as server locations from the url
## path. Another potential method could be adding appearance thresholds for certain words i.e. if B8VAgxHCYAETD6L.jpg does not
## appear more than once, then remove it from our pool. There may be some meaningful information lost when removing the
## url. Nearly all tweets have embedded links for cnn.com, so keeping those will not help us better classify/differentiate the
## tweets. However, some are for pbs, instagram, etc. so could prove meaningful as a differentiator for clustering.
data[2] = data[2].replace('(https?:\/\/)(?:([^\n ]*))?', '', regex=True)

In [283]:
tweets = data[2].to_numpy()

In [284]:
print(tweets)

['An abundance of online info can turn us into e-hypochondriacs. Or, worse, lead us to neglect getting the care we need '
 'A plant-based diet that incorporates fish may be the key to preventing colorectal cancers:  '
 "It doesn't take much to damage your hearing at a sports bar or nightclub. That's why a billion people are at risk. "
 ... 'Ann Romney talks about her experience with MS '
 "Make sure your first marathon isn't your last! "
 "Robin Roberts' cancer diagnosis "]


In [285]:
vectorizer = CountVectorizer(stop_words='english')

In [286]:
vectorizer.fit(tweets)

CountVectorizer(stop_words='english')

In [294]:
names = vectorizer.get_feature_names()
print(names)

['000', '0000', '04', '07', '08', '10', '100', '100k', '100m', '101', '102', '103', '107', '10k', '10p', '10pe', '10pet', '10pm', '11', '110', '114', '119', '11p', '11pm', '12', '120', '125', '127', '127lbs', '13', '130', '132', '14', '140', '142', '145', '145lbs', '15', '150', '153', '153lbs', '155', '158', '15th', '16', '160', '162', '17', '170', '18', '180', '184', '188', '19', '190', '1900', '1943', '1970s', '1973', '1976', '1980s', '1985', '19b', '19th', '1m', '1pet', '1pm', '1st', '1yearonmars', '20', '200', '2000', '2009', '200k', '2010', '2011', '2012', '2013', '2014', '2015', '2020science', '2030', '2050', '20x', '21', '214lbs', '21d', '22', '220', '220f', '222', '224', '229', '23', '235', '23andme', '24', '245', '25', '250', '255', '26', '260', '265', '270', '275', '276', '28', '280', '282', '29', '293', '2m', '2nd', '2ndclassctzn', '2x', '30', '300', '30p', '30pm', '30th', '319', '32', '33', '330', '35', '350', '36', '37', '38', '3d', '3dprinted', '3p', '3pet', '3s', '40', '

C:\Users\kvron\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [288]:
features = vectorizer.transform(tweets)
features

<4045x7554 sparse matrix of type '<class 'numpy.int64'>'
	with 31043 stored elements in Compressed Sparse Row format>

In [289]:
featurearray = features.toarray()

In [290]:
print(featurearray)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [291]:
df = pd.DataFrame(featurearray, columns=names);

In [293]:
df.to_csv(r'../outputdata/featurematrix.csv', index=None)